In [5]:
%%html
<style>
.box_style0{
    height: auto;
    background-color:black;
    background-image: url('img/125611494287978_registered_tex.jpg');
    background-size: 100px;
}
.box_style1{
    height: auto;
    background-color:black;
    background-image: url('img/125611494690178_registered_tex.jpg');
    background-size: 100px;
}
.box_style2{
    height: auto;
    background-color:black;
    background-image: url('img/125611494277906_registered_tex.jpg');
    background-size: 100px;
}
.box_style3{
    height: auto;
    background-color:black;
    background-image: url('img/125611494878291_registered_tex.jpg');
    background-size: 100px;
}
.box_style4{
    height: auto;
    background-color:black;
    background-image: url('img/125611487366942_registered_tex.jpg');
    background-size: 100px;
}
.box_style5{
    height: auto;
    background-color:black;
    background-image: url('img/125611494278283_registered_tex.jpg');
    background-size: 100px;
}
.box_style6{
    height: auto;
    background-color:black;
    background-image: url('img/125611495138902_registered_tex.jpg');
    background-size: 100px;
}
.box_style7{
    height: auto;
    background-color:black;
    background-image: url('img/125611494541823_registered_tex.jpg');
    background-size: 100px;
}
.tdw_box_style0{
    height: auto;
    background-color:black;
    background-image: url('img/img_-1.0_0.8_3.9_210_abandoned_factory.png');
    background-size: 200px;
}
.tdw_box_style1{
    height: auto;
    background-color:black;
    background-image: url('img/img_-0.8_0.4_-4.0_30_iceland_beach.png');
    background-size: 200px;
}
.tdw_box_style2{
    height: auto;
    background-color:black;
    background-image: url('img/img_-0.7_0.4_-1.5_90_suburb_scene_2018.png');
    background-size: 200px;
}
.tdw_box_style3{
    height: auto;
    background-color:black;
    background-image: url('img/img_-0.1_0.6_3.2_240_tdw_room.png');
    background-size: 200px;
}
</style>

In [6]:
import os
import sys

import socket
import random
import json
import time

from IPython.display import Video

def run_teach(txt_actions, txt_duration):
    print ("++ Executing TEACH with custom input")
    call_teach_txt = f"cd teach_gen/teach/;python interact_teach.py \
                    folder='ckpts/teach' \
                    output='out_samples/action_teach' \
                    texts='[{txt_actions}]' \
                    durs='[{txt_duration}]'"

    os.system(call_teach_txt)
    print ("+++ Teach execution done")
    
def run_smpl_to_fbx():
    print ("++ Converting SMPL-X to FBX")
    call_fbx_converter_txt = "cd teach_gen;~/anaconda/envs/teach/bin/python convert_to_FBX_py37.py"
    os.system(call_fbx_converter_txt)
    print ("+++ SMPL-X to FBX done")
    
def send_fbx_to_server(clothe, scene, length):
    print ("-----")
    print (clothe, scene, length)
    print ("-----")
    HOSTNAME = ""
    PORT = 10801

    # # Send FBX data to server
    path_to_fbx = "teach_gen/teach_to_fbx_outdir/teach_fbx_0.fbx"
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((HOSTNAME, PORT))
    # send FBX
    with open(path_to_fbx, "rb") as f:
        data_fbx = f.read()
    s.sendall(data_fbx)
    # now I can close the connection
    s.shutdown(socket.SHUT_RDWR)
    s.close()
    print("Sent FBX to server.")
    
    time.sleep(2)
    PORT = 10800
    # # Reopen to send additiona information
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((HOSTNAME, PORT))
    # send additional data to set clothe, select scene and length of clip
    data = json.dumps({"clothe": clothe, "scene": scene, "length": length})
    s.sendall(bytes(data, encoding="utf-8"))
    # s.send(data.encode())  
    # now I can close the connection
    s.shutdown(socket.SHUT_RDWR)
    s.close()
    print("Sent additional information.")
    
def get_mp4_from_server():
    HOSTNAME = "0.0.0.0"
    PORT = 10802

    ## now open socket in same host/port to recieve result
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((HOSTNAME, PORT))
    s.listen()

    output_name_tmp = ""

    while 1:
        print("Waiting for connections.")
        conn, addr = s.accept()
        with open(f"result_{addr[1]}.mp4", "ab+") as f:
            print("Connected to addr:", addr)
            while 1:
                data = conn.recv(1024)
                if not data:
                    break
                f.write(data)
            conn.close()
        print(f"Saved video output from {addr} at 'result_{addr[1]}.mp4'")
        output_name_tmp = f'result_{addr[1]}.mp4'

        break

    s.shutdown(socket.SHUT_RDWR)
    s.close()
    
    return output_name_tmp

In [7]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, VBox, HBox, Label, Box, Text
from IPython.display import HTML
from IPython.display import clear_output
import glob

all_imgs_clothes = glob.glob("img/*.jpg")
all_imgs_scenes = glob.glob("img/*.png")

output_name_tmp = ""
chosen_clothe = all_imgs_clothes[0]
chosen_scene = all_imgs_scenes[0]

def on_clothe_selection(b):
    global chosen_clothe
    chosen_clothe = b.description
    carousel_selected_show_clothe.value = chosen_clothe.split("/")[-1].split(".jpg")[0]
    
def on_tdw_scene_selection(b):
    global chosen_scene
    chosen_scene = b.description
    carousel_selected_show_scene.value = chosen_scene.split("/")[-1].split(".png")[0]

def css_btn_clothe_styling():
    styles = open("clothe_btn_style.css", "r").read()
    return HTML(styles)

# load cell style for buttons and images
css_btn_clothe_styling()
item_layout = Layout(height='100px', min_width='10px')
txt_layout = widgets.Layout(width='auto', height='40px') 

actions_text = widgets.Text(
    value='wave right hand, wave left hand, move backwards',
    placeholder='type actions separated by comma',
    description='String:',
    disabled=False, layout=txt_layout
)
display(actions_text)

duration_text = widgets.Text(
    value='5, 5, 5',
    placeholder='type duration of actions separated by comma',
    description='String:',
    disabled=False, layout=txt_layout
)
display(duration_text)

items_clothes = []
for ii, tt in enumerate(all_imgs_clothes):
    b = Button(layout=item_layout, description=tt, button_style='warning', tooltip=tt) 
    b.add_class(f"box_style{ii}")
    b.on_click(on_clothe_selection)
    items_clothes.append(b)

box_layout = Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='900px',
                    height='',
                    flex_flow='row',
                    display='flex')
carousel_selected_show_clothe = Text(value=chosen_clothe, disabled=True, layout=txt_layout)
carousel_clothes = Box(children=items_clothes, layout=box_layout)
vb1 = VBox([Label('Select one clothe:'), carousel_selected_show_clothe, carousel_clothes])

####

items_scenes = []
for ii, tt in enumerate(all_imgs_scenes):
    b = Button(layout=item_layout, description=tt, button_style='warning', tooltip=tt) 
    b.add_class(f"tdw_box_style{ii}")
    b.on_click(on_tdw_scene_selection)
    items_scenes.append(b)

box_layout = Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='600px',
                    height='',
                    flex_flow='row',
                    display='flex')
carousel_scenes = Box(children=items_scenes, layout=box_layout)
carousel_selected_show_scene = Text(value=chosen_scene, disabled=True, layout=txt_layout)
vb2 = VBox([Label('Select one scene:'), carousel_selected_show_scene, carousel_scenes])

display(VBox([vb1, vb2]))

button = widgets.Button(description="TEACH to TDW!")
display(button)


def on_button_clicked(b):
    global output_name_tmp
    clear_output()
    display(actions_text)
    display(duration_text)
    display(VBox([vb1, vb2]))
    display(button)
    print ("Start running!")
    run_teach(actions_text.value, duration_text.value)
    run_smpl_to_fbx()
    send_fbx_to_server(chosen_clothe, chosen_scene, 15)
    output_name_tmp = get_mp4_from_server()
    display(Video(output_name_tmp, width=1024, height=576))

button.on_click(on_button_clicked)

Text(value='walk, stop, wave right hand, wave left hand', description='String:', layout=Layout(height='40px', …

Text(value='5, 2, 5, 5', description='String:', layout=Layout(height='40px', width='auto'), placeholder='type …

Button(description='TEACH to TDW!', style=ButtonStyle())

Start running!
++ Executing TEACH with custom input


Global seed set to 1234


[21/11/22 03:06:03][__main__][INFO] - Loading model
[21/11/22 03:06:04][torch.distributed.nn.jit.instantiator][INFO] - Created a temporary directory at /tmp/tmp5a24bd24
[21/11/22 03:06:04][torch.distributed.nn.jit.instantiator][INFO] - Writing /tmp/tmp5a24bd24/_remote_module_non_sriptable.py
[21/11/22 03:06:07][__main__][INFO] - Model 'teach' loaded
[21/11/22 03:06:10][__main__][INFO] - Model weights restored


Global seed set to 0


+++ Teach execution done
++ Converting SMPL-X to FBX
Converted TEACH output to FBX in teach_to_fbx_outdir/teach_fbx_0.fbx
+++ SMPL-X to FBX done
-----
img/125611494878291_registered_tex.jpg img/img_-0.8_0.4_-4.0_30_iceland_beach.png 15
-----
Sent FBX to server.
Sent additional information.
Waiting for connections.
Connected to addr: ('9.47.227.246', 55258)
Saved video output from ('9.47.227.246', 55258) at 'result_55258.mp4'


In [11]:
display(Video("result_37962.mp4", width=1024, height=576))

In [12]:
display(Video("result_43548.mp4", width=1024, height=576))

In [18]:
display(Video("result_52968.mp4", width=1024, height=576))

In [13]:
display(Video("result_45024.mp4", width=1024, height=576))

In [15]:
display(Video("result_47264.mp4", width=1024, height=576))